In [1]:
from bs4 import BeautifulSoup as bs
import requests
import datetime, time
import pickle

import pandas as pd
import numpy as np

import os
import sys
import io
import shutil

import matplotlib.pyplot as plt

In [2]:
# !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
import cloudscraper
scraper = cloudscraper.create_scraper()

In [3]:
# # !pip install cfscrape # 403 forbidden, cloudflare error을 해결하기 위한 모듈
# import cfscrape
# scraper = cfscrape.create_scraper()
# # 이후 403 error이 발생한 곳에는 requests 대신 scraper 사용

In [4]:
module_path = os.path.abspath(os.path.join('.')) 
sys.path.append(module_path+"\\data\\constant")

from constants import US_SECTOR_LIST

In [5]:
headers = {'User-Agent': 'Mozilla/6.0 (Macintosh; Intel Mac OS X 10_11_5) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

headers = {'Referer': 'https://kr.investing.com/',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'}

In [6]:
def concat_df(df_o, df, dup_col, sort_col):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=[dup_col], keep='last', inplace=True) # dup_col 중첩제거 기준 컬럼 이름: "time", "date" 등
#     df_o.drop_duplicates(subset=[dup_col], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[sort_col]], inplace=True) # sort_col 정렬 기준 컬럼 번호
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

## 시간별 시세

In [7]:
# 특정일의 시간별 시세 취득을 위한 페이지별 table 데이터 취득. page는 날짜별로 약 40쪽. 09:00:00부터 1분간격으로 15:58:00 까지.
def get_piece_time_price(url_t):
    res = scraper.get(url_t, headers=headers)
    class_name = 'type2'
    df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
    
    df = df.dropna(axis=0) # delete nan rows

    df.columns = ['time', 'price', 'change', 'sell', 'buy', 'volume', 'change_volumn'] # rename column
    
    df['change'] = df['change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                      else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                            else int(x[4:].replace(',',''))))  # convert characters to int
    
    df = df[['time', 'price', 'change', 'volume']]  # delete unnecessary columns
    
    df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())  # convert characters to datetime objet
    df['price'] = df['price'].astype(int)
    df['volume'] = df['volume'].astype(int)

    return df

In [8]:
# 특정일의 시간대별 가격 40쪽을 한개의 df로 묶는 기능
def get_time_price(url_base_t, code_com, collect_date):
    
    page_num = 1
    
    # make first data frame
    page = str(page_num)
    
    collect_date_time = collect_date + '170000' # 장종료후 16시 00분 00초에 시간별 시세 추출
    
    url = url_base_t + '?code=' + code_com + '&thistime=' + collect_date_time + '&page=' + page
    df_base = get_piece_time_price(url)
    
    page_num = page_num + 1
    
    while True:
        page = str(page_num)
        
        url = url_base_t + '?code=' + code_com + '&thistime=' + collect_date_time + '&page=' + page
        
        df_p = get_piece_time_price(url)
    
        df_base = concat_df(df_base, df_p, 'time', 0)
        # print('page_num', page_num)
        
        if df_p['time'].iloc[-1] == datetime.time(9, 00):
            break
    
        page_num = page_num + 1
    
    df_base['date'] = datetime.datetime.strptime(collect_date, '%Y%m%d') # insert column with collecting date
    df_base = df_base[['date', 'time', 'price', 'change', 'volume']]  

    return df_base

In [9]:
url_base = 'https://finance.naver.com/item/sise_time.naver'  # sise_date

code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], 
        '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code_list = list(code_dic.items())
# code_company_name = code_list[0]
# code = code_company_name[0] # 취득을 원하는 회사 주식 코드

c_date = ['20240620', '20240621', '20240624'] # 취득이 필요한 날짜 리트트
# collect_date = c_date[2]

for i, (code, company_name) in enumerate(code_dic.items()):
    for collect_date in c_date:
        df_collect = get_time_price(url_base, code, collect_date)
        # add logic to save date for each date for each company

In [11]:
df_collect.tail()

,date,time,price,change,volume
386,2024-06-24,15:50:00,85900,-400,171774
387,2024-06-24,15:51:00,85900,-400,171815
388,2024-06-24,15:52:00,85900,-400,171834
389,2024-06-24,15:53:00,85900,-400,171844
390,2024-06-24,15:54:00,85900,-400,171845


## 일별시세

In [12]:
# 일자별 주식 데이터를 페이지별로 10개씩 취득
def get_piece_date_price(url_d):
    res = scraper.get(url_d, headers=headers)
    class_name = 'type2'
    df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
    
    df = df.dropna(axis=0) # delete nan rows

    df.columns = ['date', 'close', 'close_change', 'open', 'high', 'low', 'volume'] # rename column

    df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y.%m.%d')) # convert character to datetime object

    df['close_change'] = df['close_change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                  else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                        else int(x[4:].replace(',',''))))  # convert characters to int
    
    df = df[['date', 'open', 'high', 'low', 'close', 'close_change', 'volume']]  # rearrange columns
    
    df['open'] = df['open'].astype(int)
    df['high'] = df['high'].astype(int)
    df['low'] = df['low'].astype(int)
    df['close'] = df['close'].astype(int)
    df['volume'] = df['volume'].astype(int)

    return df

In [13]:
# 10개씩의 일자별 데이터를 원하는 일자부터 현재일자까지 합하어 취득
def get_date_price(url_base_d, code_com):
    
    page_num = 1
    
    # make first data frame
    page = str(page_num)
    
    url_date = url_base_d + '?code=' + code_com + '&page=' + page
    
    df_base = get_piece_date_price(url_date)
    
    page_num = page_num + 1
    
    while True:
        page = str(page_num)
        
        url_date = url_base_d + '?code=' + code_com + '&page=' + page
        
        df_p = get_piece_date_price(url_date)
    
        df_base = concat_df(df_base, df_p, 'date', 0)
        # print('page_num', page_num)

        startdate_str = '2021/12/23 00:00:00' # 데이터 수집 시작 일자
        # startdate_str = '2024/5/29 00:00:00'
        startdate = datetime.datetime.strptime(startdate_str, '%Y/%m/%d %H:%M:%S')

        if (startdate == df_base['date']).any():
            break
        # print("page_num", page_num)
        # print(df_p)
        page_num = page_num + 1
    
    # df_base['date'] = datetime.datetime.strptime(collect_date, '%Y.%m.%d') # insert column with collecting date
    # df_base = df_base[['date', 'time', 'price', 'change', 'volume']]  

    return df_base

In [14]:
url_base = 'https://finance.naver.com/item/sise_day.naver'  # sise_day

code_dic = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'], 
        '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}

code_dic = {'005930': ['삼성전자', 'sec'],}

for i, (code, company_name) in enumerate(code_dic.items()):
    df_collect = get_date_price(url_base, code)
    # add logic to save date for each date for each company

In [15]:
df_collect.head()

,date,open,high,low,close,close_change,volume
0,2021-12-17,76800,78000,76800,78000,200,13108479
1,2021-12-20,77600,77800,76800,77100,-900,11264375
2,2021-12-21,77900,78300,77500,78100,1000,14245298
3,2021-12-22,78900,79400,78800,79400,1300,17105892
4,2021-12-23,79800,80000,79300,79900,500,13577498


## 여기까지

### 아래는 지울 것

In [ ]:
url_date

In [ ]:
df

In [ ]:
date	open	high	low	close	close_cr	vol
df_get = df[['date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']]

In [ ]:
# df.columns = ['date', 'time', 'price', 'change', 'change(%)', 'amount', 'volume']

In [ ]:
df.head()

In [ ]:
df.iloc[2,2]= char1

In [ ]:
df.head()

In [ ]:
df[df['change'].str[:2] == '상승']

In [ ]:
df['change'] = df['change'].apply(lambda x: int(x[2:]) if x[:2] == '보합' 
                                  else (-int(x[4:].replace(',','')) if x[:2] == '하락' 
                                        else int(x[4:].replace(',',''))))

In [ ]:
df.head()

In [ ]:
df = df[['date', 'time', 'price', 'change', 'volume']]

In [ ]:
df

In [ ]:
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())

In [ ]:
df['time'][1]

In [ ]:
char1 = df.iloc[2,2]

In [ ]:
len(char)

In [ ]:
char[:2]

In [ ]:
if char[:2] == '상승':
    new_char = int(char[4:])
    print("1")
elif char[:2] == '하락':
    new_char = char[4:]
    print("2")
elif char[:2] == '보합':
    new_char = int(char[2:])
    print("3")

In [ ]:
new_char

In [ ]:
char[2:]

In [ ]:
new_char*df.iloc[0,1]

In [ ]:
rdata.reset_index('Date', inplace=True)
rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()
rdata = rdata[(rdata['Date'] <= enddate) & (rdata['Date'] >= startdate)] 
rdata['temp'] = rdata['Close'].shift(1)
val_temp = (rdata['Close'] - rdata['temp'])/rdata['temp']*100
rdata[f'{col_name}_cr'] = val_temp.map("{:.2f}%".format)
rdata = rdata[['Date', 'Close', 'Open', 'High', 'Low', 'Volume', f'{col_name}_cr']] # 필요한 column만 남김
rdata.columns = ['date', f'{col_name}', 'open', 'high', 'low', 'volume', f'{col_name}_cr'] # column이름 통일
rdata.reset_index(drop=True, inplace=True) # index번호를 0부터 재정리

In [ ]:
df.tail()

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
def make_pickle(df, pkl_name):
    pkl_directory = 'data/common_pkl/'
    try:
        if not os.path.exists(pkl_directory):
            os.makedirs(pkl_directory)
    except OSError:
        print("Error: Failed to create the directory.")

    # 데이터 저장: ../data/spx.pkl
    df.to_pickle(pkl_directory+pkl_name)
    df.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))

In [ ]:
def read_pickle(pkl_name):
# 데이터 로드
    pkl_directory = 'data/common_pkl/'
    df = pd.read_pickle(pkl_directory+pkl_name)
    
    return df

In [ ]:
def get_data(url, column):
           
    count = 0
    while True:
        try :
            res = scraper.get(url, headers=headers)
            print("t1", res.status_code)
#             res = requests.get(url, headers=headers)
#             class_name = 'w-full text-xs leading-4 overflow-x-auto freeze-column-w-1'
# 위의 class_name 명칭은 수시로 바뀌므로 새롭게 수정해 주어야 함.
            class_name = 'freeze-column-w-1 w-full overflow-x-auto text-xs leading-4'
            # df = pd.read_html(res.text, attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
            df = pd.read_html(io.StringIO(str(res.text)), attrs={"class": class_name}, flavor=["lxml", "bs4"])[0]
#             res = scraper.get(url, headers=headers)  # 변경전
#             df = pd.read_html(res.text, attrs={"id": "curr_table"}, flavor=["lxml", "bs4"])[0]
            break
        except:
            try:
                # res = scraper.get(url, headers=headers)
                res = requests.get(url, headers=headers)
                print("t2", res.status_code)
                # df = pd.read_html(res.text, attrs={"data-test": "historical-data-table"}, flavor=["lxml", "bs4"])[0]
                df = pd.read_html(io.StringIO(str(res.text)), attrs={"data-test": "historical-data-table"}, flavor=["lxml", "bs4"])[0]
                break
            except:
                pass
        finally:
            time.sleep(1)
            print("t3")
            count += 1
            if count > 5 :
                raise ValueError('The url request is delaying')
                break           

    df.columns = column
    correct_date_format(df)
    df.sort_values(by=[df.columns[0]], inplace=True)
    df.index = np.arange(0, len(df))  # 일련 번호 오름차순으로 재 설정

    return df

In [ ]:
def concat_df(df_o, df):
    df_o = pd.concat([df_o, df], ignore_index=True)
    df_o.drop_duplicates(subset=['date'], keep='last', inplace=True)
#     df_o.drop_duplicates(subset=['date'], keep='first', inplace=True)
    df_o.sort_values(by=[df_o.columns[0]], inplace=True)
    df_o.index = np.arange(0, len(df_o))  # 일련 번호 오름차순으로 재 설정
    return df_o

In [ ]:
def update_pickle(df, pkl_name):
    df_o = read_pickle(pkl_name)
    try:  # convert timestamp to datetime.datetime.date
        df_o['date'] = df_o['date'].apply(lambda x: datetime.datetime.date(x))
    except:
        pass
    df_o = concat_df(df_o, df)
    
    make_pickle(df_o, pkl_name)

In [ ]:
# reformat data from yfinance
def reformat_df(df, col_name):
    df = df.reset_index('Date')
    df.drop(labels='Adj Close', axis=1, inplace=True)
    df = df[['Date', 'Close', 'Open', 'High', 'Low', 'Volume']]
    df.columns = ['date', col_name, 'open', 'high', 'low', 'volume']
    df['temp'] = df[col_name].shift(1)
    df[col_name+'_cr'] = ((df[col_name] - df['temp'])/df['temp']*100).apply(lambda x: f'{x:.2f}%')
    df.drop(labels='temp', axis=1, inplace=True)
    return df

In [ ]:
def get_ticker_data(ticker, startdate, enddate, col_name):
    while True:
        ydata = yf.Ticker(ticker)
        rdata = ydata.history(period="5y") # 오늘부터 3년치
        if len(rdata) <= 100: # 인터넷 속도로 인한 데이터 취득이 되지 않았을 때 임의의 수(100)으로 비교
            pass
        else:
            break
    rdata.reset_index('Date', inplace=True)
    rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()
    rdata = rdata[(rdata['Date'] <= enddate) & (rdata['Date'] >= startdate)] 
    rdata['temp'] = rdata['Close'].shift(1)
    val_temp = (rdata['Close'] - rdata['temp'])/rdata['temp']*100
    rdata[f'{col_name}_cr'] = val_temp.map("{:.2f}%".format)
    rdata = rdata[['Date', 'Close', 'Open', 'High', 'Low', 'Volume', f'{col_name}_cr']] # 필요한 column만 남김
    rdata.columns = ['date', f'{col_name}', 'open', 'high', 'low', 'volume', f'{col_name}_cr'] # column이름 통일
    rdata.reset_index(drop=True, inplace=True) # index번호를 0부터 재정리
    return rdata

In [ ]:
# startdate = datetime.datetime(2021,12,25)
startdate = datetime.date(2021,12,25)
# enddate = datetime.datetime(2023,3,23)
enddate = datetime.date.today() + datetime.timedelta(days=2)

In [ ]:
# # spx = pdr.get_data_yahoo('^SPX', start=startdate, end=enddate)
# # df = reformat_df(spx, 'spx')
# # make_pickle(df, 'spx.pkl')

# # spx는 yahoo에서 최근 하루만 제공하기 때문에 investing을 계속 사용함
# spx_url = 'https://kr.investing.com/indices/us-spx-500-historical-data'
# spx = ['date', 'spx', 'open', 'high', 'low', 'volume', 'spx_cr']
# pkl_name = 'spx.pkl'
# df = get_data(spx_url,spx)

# update_pickle(df, pkl_name)

In [ ]:
spx = get_ticker_data('^SPX', startdate, enddate, 'spx')
make_pickle(spx, 'spx.pkl')

In [ ]:
dji = get_ticker_data('^DJI', startdate, enddate, 'dji')
make_pickle(dji, 'dji.pkl')

# 시간대별로 막혀 있어서 아래 것은 사용하지 않음 (미국 개장시간대에서만 열림)
# dji = pdr.get_data_yahoo('^DJI', start=startdate, end=enddate)
# df = reformat_df(dji, 'dji')
# make_pickle(df, 'dji.pkl')

# 아래 investing은 vol자료에 차이가 있어 사용하지 않음.
# dji_url = 'https://www.investing.com/indices/us-30-historical-data'
# dji = ['date', 'dji', 'open', 'high', 'low', 'volume', 'dji_cr']
# pkl_name = 'dji.pkl'
# df = get_data(dji_url,dji)

# update_pickle(df, pkl_name)

In [ ]:
ixic = get_ticker_data('^IXIC', startdate, enddate, 'ixic')
make_pickle(ixic, 'nas.pkl')

# ixic = pdr.get_data_yahoo('^IXIC', start=startdate, end=enddate)
# df = reformat_df(ixic, 'ixic')
# make_pickle(df, 'nas.pkl')

# nas_url = 'https://kr.investing.com/indices/nasdaq-composite-historical-data'
# ixic = ['date', 'ixic', 'open', 'high', 'low', 'volume', 'ixic_cr']
# pkl_name = 'nas.pkl'
# df = get_data(nas_url,ixic)

# update_pickle(df, pkl_name)

In [ ]:
sox = get_ticker_data('^SOX', startdate, enddate, 'sox')
make_pickle(sox, 'sox.pkl')

# sox = pdr.get_data_yahoo('^SOX', start=startdate, end=enddate)
# df = reformat_df(sox, 'sox')
# make_pickle(df, 'sox.pkl')

# sox_url = 'https://kr.investing.com/indices/phlx-semiconductor-historical-data'
# sox = ['date', 'sox', 'open', 'high', 'low', 'volume', 'sox_cr']
# pkl_name = 'sox.pkl'
# df = get_data(sox_url,sox)

# update_pickle(df, pkl_name)

In [ ]:
vix = get_ticker_data('^VIX', startdate, enddate, 'vix')
make_pickle(vix, 'vix.pkl')

# vix = pdr.get_data_yahoo('^VIX', start=startdate, end=enddate)
# df = reformat_df(vix, 'vix')
# make_pickle(df, 'vix.pkl')

# vix_url = 'https://kr.investing.com/indices/volatility-s-p-500-historical-data'
# vix = ['date', 'vix', 'open', 'high', 'low', 'volume', 'vix_cr']
# pkl_name = 'vix.pkl'
# df = get_data(vix_url,vix)

# update_pickle(df, pkl_name)

In [ ]:
kor_bond_10yr_url = 'https://kr.investing.com/rates-bonds/south-korea-10-year-bond-yield-historical-data'
kor_10yr = ['date', 'bond_kor_10', 'open', 'high', 'low', 'bond_kor_10_cr']
pkl_name = 'kor_10yr_bond.pkl'
df = get_data(kor_bond_10yr_url,kor_10yr)

update_pickle(df, pkl_name)

In [ ]:
url = 'https://kr.investing.com/rates-bonds/south-korea-10-year-bond-yield-historical-data'
res = requests.get(url, headers=headers)
print(res.content.decode('utf-8'))

In [ ]:
kor_bond_2yr_url = 'https://kr.investing.com/rates-bonds/south-korea-2-year-bond-yield-historical-data'
kor_2yr = ['date', 'bond_kor_2', 'open', 'high', 'low','bond_kor_2_cr']
pkl_name = 'kor_2yr_bond.pkl'
df = get_data(kor_bond_2yr_url,kor_2yr)

update_pickle(df, pkl_name)

In [ ]:
krw_rate_url = 'https://kr.investing.com/currencies/usd-krw-historical-data'
krw_rate = ['date', 'krw', 'open', 'high', 'low', 'vol', 'krw_cr']
pkl_name = 'krw_rate.pkl'
df = get_data(krw_rate_url,krw_rate)

update_pickle(df, pkl_name)

#### get append cpi

In [ ]:
cpi_url = 'https://www.investing.com/economic-calendar/cpi-733'
cpi_column = ['date', 'time', 'cpi', 'cpi_anticipated', 'cpi_previous', 'none']
pkl_name = 'cpi.pkl'

In [ ]:
res = scraper.get(cpi_url, headers=headers)
# res = requests.get(cpi_url, headers=headers)
# df = pd.read_html(res.text, flavor=["lxml", "bs4"])[0]
df = pd.read_html(io.StringIO(str(res.text)), flavor=["lxml", "bs4"])[0]
df.columns = cpi_column
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))

df = df[['date', 'cpi', 'cpi_anticipated', 'cpi_previous']]

try: # convert timestamp to datetime.datetime.date
    df['date'] = df['date'].apply(lambda x: datetime.datetime.date(x))
except:
    pass

df.sort_values(by=['date'], inplace=True)

update_pickle(df, pkl_name)

#### get append fear and greed
##### 2020년 9월 21일부터  2021년 1월 21일까지 데이터는 이상 데이터 로 나중에 수정해야 함.

In [ ]:
import pytz, json

In [ ]:
def convert_timestamp_to_date(x):
    dt = datetime.datetime.fromtimestamp(x / 1000, tz=pytz.utc) # UTC에서 변환 불필요.
#     tzone = pytz.timezone('US/Eastern')
#     tzone = pytz.timezone('Asia/Seoul')
#     loc_dt = dt.astimezone(tzone)
    loc_dt = dt
    return loc_dt.date()

In [ ]:
# 과거 데이터 (fear_greed_old_to_20200918.pkl) 에 rating column을 추가한 내용
# 한번만 사용하고 이후 사용하지 않음

def convert_to_rating(x):
    if x < 25 :
        rating = 'extreme fear'
    elif x < 45 :
        rating = 'fear'
    elif x < 55 :
        rating = 'neutral'
    elif x < 75 :
        rating = 'greed'
    elif x <= 100 :
        rating = 'extreme greed'

    return rating

In [ ]:
today = datetime.date.today()
today_p = today.strftime('%Y%m%d')
diff_days = datetime.timedelta(days=40)
today = today - diff_days
start_date = today.strftime('%Y-%m-%d')  # 30일전부터 자료 수집

In [ ]:
url = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
pkl_name = 'fear_greed.pkl'
# start_date = '2020-07-15'

In [ ]:
r = requests.get("{}/{}".format(url, start_date), headers=headers)
data = r.json()

fg_data = data['fear_and_greed_historical']['data']
df = pd.DataFrame(fg_data)

df.columns = ['date', 'fg_index', 'rating']
df['date'] = df['date'].apply(lambda x: convert_timestamp_to_date(x))
df['fg_index'] = df['fg_index'].apply(lambda x: round(x))

df.sort_values(by=[df.columns[0]], inplace=True)
df.index = np.arange(0, len(df))  # 일련 번호 오름차순으로 재 설정
df.drop_duplicates(subset=['date'], inplace=True) 

In [ ]:
df_o = read_pickle(pkl_name)
df_o['date'] = df_o['date'].apply(lambda x : datetime.datetime.strftime(x, "%Y-%m-%d"))
df_o['date'] = df_o['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
try:  # convert timestamp to datetime.datetime.date
    df_o['date'] = df_o['date'].apply(lambda x: datetime.datetime.date(x))
except:
    pass

df_o = concat_df(df_o, df)

# 분석시 제외. 추후 보강 2023년 8월 이후에 추가 할지 결정해야 함.
make_pickle(df_o, pkl_name)

#### get and append gold price

In [ ]:
# gold_url = 'https://www.usagold.com/daily-gold-price-history/'
# pkl_name = 'gold.pkl'

# res = requests.get(gold_url, headers=headers)
## df = pd.read_html(res.text, flavor=["lxml", "bs4"])
# df = pd.read_html(io.StringIO(str(res.text)), flavor=["lxml", "bs4"])
# df = df[0].drop(0) # delete empty first row
# df.columns = ['date', 'gold']
# df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%d %b %Y"))
# df.sort_values(by=['date'], inplace=True)
# df.drop_duplicates(subset=['date'], inplace=True) 

# # 분석시 제외. 추후 보강 현재는 빠짐.
# update_pickle(df, pkl_name)

In [ ]:
gold = get_ticker_data('GC=F', startdate, enddate, 'gold')
make_pickle(gold, 'gold.pkl')

# # Gold Apr 23 (GC=F)
# df = pdr.get_data_yahoo('GC=F', start=startdate, end=enddate)
# df = reformat_df(df, 'gold')
# make_pickle(df, 'gold.pkl')

### fed 금리 get append

In [ ]:
interest_url = 'https://www.investing.com/economic-calendar/interest-rate-decision-168/'
interest_column = ['date', 'time', 'fed_rate', 'fed_rate_fore', 'fed_rate_prev', 'none']
pkl_name = 'fed_rate.pkl'

In [ ]:
res = scraper.get(interest_url, headers=headers)
# res = requests.get(interest_url, headers=headers)
# df = pd.read_html(res.text, flavor=["lxml", "bs4"])[0]
df = pd.read_html(io.StringIO(str(res.text)), flavor=["lxml", "bs4"])[0]
df.columns = interest_column

df.replace(np.nan, '', inplace=True)

df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True) 
# convert from timestampe to datetime
df['date'] = df['date'].apply(lambda x: datetime.datetime.date(x)) 

In [ ]:
# 매일매일의 데이터가 없어서 분석시 제외. 추후 보강 현재는 빠짐.
update_pickle(df, pkl_name)

### 한국은행 금리 get append

In [ ]:
kor_url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'
pkl_name = 'bok_rate.pkl'

In [ ]:
# res = requests.get(kor_url, headers=headers, verify=certifi.where())
res = requests.get(kor_url, headers=headers)
df = pd.read_html(io.StringIO(str(res.text)), attrs = {'class': 'fixed'}, flavor=["lxml", "bs4"])[0]

In [ ]:
df.columns=  ['release_yr', 'release_date', 'bok_rate']
df_date_temp = df['release_yr'].astype('str')+df['release_date']
df['date'] = df_date_temp.apply(lambda x : datetime.datetime.strptime(x, "%Y%m월 %d일"))
df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True) 

df = df[['date', 'bok_rate']] # leave only valid columns

# convert from timestampe to datetime
df['date'] = df['date'].apply(lambda x: datetime.datetime.date(x)) 

# 매일매일의 데이터가 없어서 분석시 제외. 추후 보강 현재는 빠짐.
update_pickle(df, pkl_name)

### US Sector Summary

In [ ]:
# startdate = datetime.datetime(2021,12,25)
# enddate = datetime.datetime(2023,3,23)

In [ ]:
# [ticker, shortname, filename]
us_sector_list = US_SECTOR_LIST

In [ ]:
for sector in us_sector_list:
    make_pickle(get_ticker_data(sector[0], startdate, enddate, sector[1]), sector[2])

### 여기까지 완료

In [ ]:
ticker = '^SP500-40'
ydata = yf.Ticker(ticker)
rdata = ydata.history(period="2y") # 오늘부터 2년치
rdata.reset_index('Date', inplace=True)

In [ ]:
rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()

In [ ]:
rdata.info()

In [ ]:
rdata.head()

In [ ]:
rdata['Date'] = rdata['Date'].dt.date  # datetime64 to datetime.date()
rdata = rdata[(rdata['Date'] <= enddate) & (rdata['Date'] >= startdate)] 
rdata['temp'] = rdata['Close'].shift(1)
val_temp = (rdata['Close'] - rdata['temp'])/rdata['temp']*100
rdata[f'{col_name}_cr'] = val_temp.map("{:.2f}%".format)
rdata = rdata[['Date', 'Close', 'Open', 'High', 'Low', 'Volume', f'{col_name}_cr']] # 필요한 column만 남김
rdata.columns = ['date', f'{col_name}', 'open', 'high', 'low', 'volume', f'{col_name}_cr'] # column이름 통일
rdata.reset_index(drop=True, inplace=True) # index번호를 0부터 재정리

### 아래는 삭제 예정

In [ ]:
# # S&P 500 Financials (SPSY), ^SP500-40
# # spsy_url = 'https://www.investing.com/indices/s-p-500-financial-historical-data'
# spsy = get_ticker_data('^SP500-40', startdate, enddate, 'spsy')
# make_pickle(spsy, 'spsy.pkl')

# # df = pdr.get_data_yahoo('^SP500-40', start=startdate, end=enddate)
# # df = reformat_df(df, 'spsy')
# # make_pickle(df, 'spsy.pkl')

In [ ]:
# # S&P 500 Energy (SPNY), ^GSPE
# # spny_url = 'https://www.investing.com/indices/s-p-500-energy-historical-data'
# spny = get_ticker_data('^GSPE', startdate, enddate, 'spny')
# make_pickle(spny, 'spny.pkl')

# # df = pdr.get_data_yahoo('^GSPE', start=startdate, end=enddate)
# # df = reformat_df(df, 'spny')
# # make_pickle(df, 'spny.pkl')

In [ ]:
# # spxhc_url = 'https://www.investing.com/indices/s-p-500-health-care-historical-data'
# # S&P 500 Health Care (SPXHC), ^SP500-35
# spxhc = get_ticker_data('^SP500-35', startdate, enddate, 'spxhc')
# make_pickle(spxhc, 'spxhc.pkl')

# # spxhc = pdr.get_data_yahoo('^SP500-35', start=startdate, end=enddate)
# # df = reformat_df(spxhc, 'spxhc')
# # make_pickle(df, 'spxhc.pkl')

In [ ]:
# # splrcd_url = 'https://www.investing.com/indices/s-p-500-consumer-discretionary-historical-data'
# # S&P 500 Consumer Discretionary (SPLRCD), ^SP500-25
# splrcd = get_ticker_data('^SP500-25', startdate, enddate, 'splrcd')
# make_pickle(splrcd, 'splrcd.pkl')

# # splrcd = pdr.get_data_yahoo('^SP500-25', start=startdate, end=enddate)
# # df = reformat_df(splrcd, 'splrcd')
# # make_pickle(df, 'splrcd.pkl')

In [ ]:
# # splrci_url = 'https://www.investing.com/indices/s-p-500-industrials-historical-data'
# # S&P 500 Industrials (SPLRCI), ^SP500-20
# splrci = get_ticker_data('^SP500-20', startdate, enddate, 'splrci')
# make_pickle(splrci, 'splrci.pkl')

# # splrci = pdr.get_data_yahoo('^SP500-20', start=startdate, end=enddate)
# # df = reformat_df(splrci, 'splrci')
# # make_pickle(df, 'splrci.pkl')

In [ ]:
# # splrcu_url = 'https://www.investing.com/indices/s-p-500-utilities-historical-data'
# # S&P 500 Utilities (SPLRCU), ^SP500-55
# splrcu = get_ticker_data('^SP500-55', startdate, enddate, 'splrcu')
# make_pickle(splrcu, 'splrcu.pkl')

# # splrcu = pdr.get_data_yahoo('^SP500-55', start=startdate, end=enddate)
# # df = reformat_df(splrcu, 'splrcu')
# # make_pickle(df, 'splrcu.pkl')

In [ ]:
# # splrcs_url = 'https://www.investing.com/indices/s-p-500-consumer-staples-historical-data'
# # S&P 500 Consumer Staples (SPLRCS), ^SP500-30
# splrcs = get_ticker_data('^SP500-30', startdate, enddate, 'splrcs')
# make_pickle(splrcs, 'splrcs.pkl')

# # splrcs = pdr.get_data_yahoo('^SP500-30', start=startdate, end=enddate)
# # df = reformat_df(splrcs, 'splrcs')
# # make_pickle(df, 'splrcs.pkl')

In [ ]:
# # splrct_url = 'https://www.investing.com/indices/s-p-500-information-technology-historical-data'
# # S&P 500 Information Technology (SPLRCT), ^SP500-45
# splrct = get_ticker_data('^SP500-45', startdate, enddate, 'splrct')
# make_pickle(splrct, 'splrct.pkl')

# # splrct = pdr.get_data_yahoo('^SP500-45', start=startdate, end=enddate)
# # df = reformat_df(splrct, 'splrct')
# # make_pickle(df, 'splrct.pkl')

In [ ]:
# # splrcl_url = 'https://www.investing.com/indices/s-p-500-telecom-services-historical-data'
# # S&P 500 Telecom Services (SPLRCL), ^SP500-50
# splrcl = get_ticker_data('^SP500-50', startdate, enddate, 'splrcl')
# make_pickle(splrcl, 'splrcl.pkl')

# # splrcl = pdr.get_data_yahoo('^SP500-50', start=startdate, end=enddate)
# # df = reformat_df(splrcl, 'splrcl')
# # make_pickle(df, 'splrcl.pkl')

In [ ]:
# # splrcm_url = 'https://www.investing.com/indices/s-p-500-materials-historical-data'
# # S&P 500 Materials (SPLRCM), ^SP500-15
# splrcm = get_ticker_data('^SP500-15', startdate, enddate, 'splrcm')
# make_pickle(splrcm, 'splrcm.pkl')

# # splrcm = pdr.get_data_yahoo('^SP500-15', start=startdate, end=enddate)
# # df = reformat_df(splrcm, 'splrcm')
# # make_pickle(df, 'splrcm.pkl')

In [ ]:
# # ixbk_url = 'https://www.investing.com/indices/nasdaq-bank-historical-data'
# # NASDAQ Bank (IXBK), ^BANK
# ixbk = get_ticker_data('^BANK', startdate, enddate, 'ixbk')
# make_pickle(ixbk, 'ixbk.pkl')

# # ixbk = pdr.get_data_yahoo('^BANK', start=startdate, end=enddate)
# # df = reformat_df(ixbk, 'ixbk')
# # make_pickle(df, 'ixbk.pkl')

In [ ]:
# yahoo에서는 하루만 잡힘. investing.com에서는 자료가 있으나 skip (나중에 보강)
# ixf_url = 'https://www.investing.com/indices/nasdaq-financial-100-historical-data'
# # NASDAQ Financial 100 (IXF), ^IXF
# ixf = pdr.get_data_yahoo('^IXF', start=startdate, end=enddate)
# df = reformat_df(ixf, 'ixf')
# make_pickle(df, 'ixf.pkl')

In [ ]:
# # ixfn_url = 'https://www.investing.com/indices/nasdaq-other-finance-historical-data'
# # NASDAQ Other Finance (IXFN), ^OFIN
# ixfn = get_ticker_data('^OFIN', startdate, enddate, 'ixfn')
# make_pickle(ixfn, 'ixfn.pkl')

# # ixfn = pdr.get_data_yahoo('^OFIN', start=startdate, end=enddate)
# # df = reformat_df(ixfn, 'ixfn')
# # make_pickle(df, 'ixfn.pkl')

In [ ]:
# # ixid_url = 'https://www.investing.com/indices/nasdaq-industrial-historical-data'
# # NASDAQ Industrial (IXID), ^INDS
# ixid = get_ticker_data('^INDS', startdate, enddate, 'ixid')
# make_pickle(ixid, 'ixid.pkl')

# # ixid = pdr.get_data_yahoo('^INDS', start=startdate, end=enddate)
# # df = reformat_df(ixid, 'ixid')
# # make_pickle(df, 'ixid.pkl')

In [ ]:
# # ixis_url = 'https://www.investing.com/indices/nasdaq-insurance-historical-data'
# # NASDAQ Insurance (IXIS), ^INSR
# ixis = get_ticker_data('^INSR', startdate, enddate, 'ixis')
# make_pickle(ixis, 'ixis.pkl')

# # ixis = pdr.get_data_yahoo('^INSR', start=startdate, end=enddate)
# # df = reformat_df(ixis, 'ixis')
# # make_pickle(df, 'ixis.pkl')

In [ ]:
# # ixk_url = 'https://www.investing.com/indices/nnasdaq-computer-historical-data'
# # NASDAQ Computer (IXK), ^IXCO
# ixk = get_ticker_data('^IXCO', startdate, enddate, 'ixk')
# make_pickle(ixk, 'ixk.pkl')

# # ixk = pdr.get_data_yahoo('^IXCO', start=startdate, end=enddate)
# # df = reformat_df(ixk, 'ixk')
# # make_pickle(df, 'ixk.pkl')

In [ ]:
# # ixtr_url = 'https://www.investing.com/indices/nasdaq-transportation-historical-data'
# # NASDAQ Transportation (IXTR), ^TRAN
# ixtr = get_ticker_data('^TRAN', startdate, enddate, 'ixtr')
# make_pickle(ixtr, 'ixtr.pkl')

# # ixtr = pdr.get_data_yahoo('^TRAN', start=startdate, end=enddate)
# # df = reformat_df(ixtr, 'ixtr')
# # make_pickle(df, 'ixtr.pkl')

In [ ]:
# # ixut_url = 'https://www.investing.com/indices/nasdaq-telecommunications-historical-data'
# # NASDAQ Telecommunications (IXUT), ^IXTC
# ixut = get_ticker_data('^IXTC', startdate, enddate, 'ixut')
# make_pickle(ixut, 'ixut.pkl')

# # ixut = pdr.get_data_yahoo('^IXTC', start=startdate, end=enddate)
# # df = reformat_df(ixut, 'ixut')
# # make_pickle(df, 'ixut.pkl')

In [ ]:
# # nbi_url = 'https://www.investing.com/indices/nasdaq-biotechnology-historical-data'
# # NASDAQ Biotechnology (NBI), ^NBI
# nbi = get_ticker_data('^NBI', startdate, enddate, 'nbi')
# make_pickle(nbi, 'nbi.pkl')

# # nbi = pdr.get_data_yahoo('^NBI', start=startdate, end=enddate)
# # df = reformat_df(nbi, 'nbi')
# # make_pickle(df, 'nbi.pkl')

In [ ]:
# yahoo에서는 하루만 잡힘. investing.com에서는 자료가 있으나 skip (나중에 보강)
# qnet_url = 'https://www.investing.com/indices/nasdaq-internet-historical-data'
# # NASDAQ Internet (QNET), ^QNET
# qnet = pdr.get_data_yahoo('^QNET', start=startdate, end=enddate)
# df = reformat_df(qnet, 'qnet')
# make_pickle(df, 'qnet.pkl')

In [ ]:
# bkx_url = 'https://www.investing.com/indices/kbw-bank-historical-data'
# # KBW NASDAQ Bank (BKX), ^BKX
# bkx = get_ticker_data('^BKX', startdate, enddate, 'bkx')
# make_pickle(bkx, 'bkx.pkl')

# # bkx = pdr.get_data_yahoo('^BKX', start=startdate, end=enddate)
# # df = reformat_df(bkx, 'bkx')
# # make_pickle(df, 'bkx.pkl')